# 問題
コーパスにおける名詞の出現頻度を求め、出現頻度の高い20語とその出現頻度を表示せよ。

In [3]:
import pandas as pd
import re
import html

# ----------------------------
# マークアップ除去関数
# ----------------------------
def clean_wiki_markup(text: str) -> str:
    """
    Wikipedia記事テキストからマークアップを除去し、プレーンテキストを返す
    """
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)  # HTMLエスケープ解除

    patterns = [
        # --- 巨大ブロック除去 ---
        (r'(?m)^\s*\{\{\s*基礎情報[\s\S]*?^\s*\}\}\s*$', "", re.MULTILINE),  # 基礎情報
        (r'\{\|[\s\S]*?\|\}', "", 0),  # 表 (table)

        # --- ハットノート ---
        (r'\{\{\s*(?:otheruses|Otheruses[^|}]*)\s*\|[^}]*\}\}', "", re.IGNORECASE),

        # --- カテゴリ ---
        (r'\[\[(?:Category|カテゴリ):[^\]]+\]\]', "", re.IGNORECASE),

        # --- HTMLコメント ---
        (r'<!--[\s\S]*?-->', "", 0),

        # --- 強調記法 ---
        (r"'''''(.*?)'''''", r"\1", 0),
        (r"'''(.*?)'''", r"\1", 0),
        (r"''(.*?)''", r"\1", 0),

        # --- 内部リンク処理 ---
        (r"\[\[(?:ファイル|File|画像):[^|\]]+(?:\|[^|\]]+)*\|([^|\]=][^|\]]*)\]\]", r"\1", 0),
        (r"\[\[(?:ファイル|File|画像):([^\]|]+)\]\]", r"\1", 0),
        (r"\[\[[^|\]]+\|([^\]]+)\]\]", r"\1", 0),
        (r"\[\[([^\]|#]+)(?:#[^\]]+)?\]\]", r"\1", 0),
        (r"\[\[:[a-z0-9\-]+:[^|\]]+\|([^\]]+)\]\]", r"\1", re.IGNORECASE),
        (r"\[\[:[a-z0-9\-]+:([^\]#|]+)(?:#[^\]]+)?\]\]", r"\1", re.IGNORECASE),

        # --- テンプレート処理 ---
        (r"\{\{lang\|[a-z0-9\-]+\|([^}]+)\}\}", r"\1", re.IGNORECASE),
        (r"\{\{lang-[a-z0-9\-]+\|([^}]+)\}\}", r"\1", re.IGNORECASE),
        (r"\{\{(?:ill2?|仮リンク)\|([^|}]+)\|[^}]*\}\}", r"\1", re.IGNORECASE),
        (r"\{\{flagicon\|([^|}]+)\}\}", r"\1", re.IGNORECASE),
        (r"\{\{convert\|([^|}]+)\|[^}]*\}\}", r"\1", re.IGNORECASE),

        # --- 参照タグ ---
        (r"(?m)<ref[^>]*>.*?$", "", 0),
        (r"<ref[^>]*>.*?</ref>", "", re.DOTALL | re.IGNORECASE),
        (r"<ref[^>]*/>", "", re.IGNORECASE),
        (r"<references\s*/?>", "", re.IGNORECASE),

        # --- HTMLタグ ---
        (r"<br\s*/?>", "\n", re.IGNORECASE),
        (r"</?(?:small|sup|sub|span|div|center|nowiki|ref)[^>]*>", "", re.IGNORECASE),
        (r"<[^>]+>", "", 0),

        (r"\{\{[^{}]+\}\}", "", 0),   # 単純なテンプレート除去
        (r"\{\{", "", 0),             # 孤立 {{ を除去
        (r"\}\}", "", 0),             # 孤立 }} を除去
    ]

    # ループで繰り返し削除
    changed = True
    while changed:
        old = text
        for pat, repl, flg in patterns:
            text = re.sub(pat, repl, text, flags=(flg | re.DOTALL))
        changed = (text != old)

    # --- 後処理 ---
    text = text.replace("\\n", "\n")  # リテラル \n を改行へ
    text = re.sub(r'(?m)^\s*={2,}\s*(.*?)\s*={2,}\s*$', r'\n\1\n', text)  # 見出しを整形
    text = re.sub(r"[ \t]+", " ", text)  # 余計な空白削除
    text = re.sub(r"\n{3,}", "\n\n", text)  # 改行を整形

    return text.strip()

# ----------------------------
# データの読み込み
# ----------------------------
df = pd.read_json("Jawiki Country.json.gz", lines=True)

In [4]:
from janome.tokenizer import Tokenizer
from collections import Counter

t = Tokenizer()
freq = Counter()

for raw in df["text"]:
    cleaned = clean_wiki_markup(raw)
    for token in t.tokenize(cleaned):
        # 名詞だけに限定
        if token.part_of_speech.startswith("名詞"):
            freq.update([token.base_form])

# 上位20件
for w, c in freq.most_common(20):
    print(w, c)

年 19390
. 9551
* 9080
人 7382
- 6768
月 5740
/ 4987
国 4964
| 4737
的 4190
語 3968
= 3882
1 3759
日 3659
こと 3622
2 3127
日本 2919
世界 2872
政府 2842
3 2787


In [ ]:
import json
import gzip
import re
import MeCab
from collections import Counter


def remove_markup(text):
    # 強調マークアップの除去
    text = re.sub(r"\'{2,5}", "", text)
    # 内部リンクの除去
    text = re.sub(r"\[\[(?:[^|\]]*?\|)??([^|\]]+?)\]\]", r"\1", text)
    # 外部リンクの除去
    text = re.sub(r"\[http://[^\]]+\]", "", text)
    # HTMLタグの除去
    text = re.sub(r"<[^>]+>", "", text)
    # テンプレートの除去
    text = re.sub(r"\{\{.*?\}\}", "", text)
    return text


def analyze_word_frequency():
    # MeCabの初期化
    mecab = MeCab.Tagger("-Owakati")
    # 単語の出現頻度をカウントするためのCounter
    word_counter = Counter()

    # gzipファイルを読み込む
    with gzip.open("Jawiki Country.json.gz", "rt", encoding="utf-8") as f:
        print("行単位で読み込まれている。次のfor文で1行ずつ取り出している。dfよりも軽い。;",f)
        for line in f:
            sentence = json.loads(line)
            text = sentence["text"]

            # マークアップを除去
            text = remove_markup(text)

            node = mecab.parseToNode(text)
            while node:
              if node.feature.startswith("名詞"):  
                  word_counter[node.surface] += 1
              node = node.next 

    # 出現頻度の高い20語を表示
    for word, count in word_counter.most_common(20):
        print(f"{word}: {count}")


analyze_word_frequency()

行単位で読み込まれている。次のfor文で1行ずつ取り出している。dfよりも軽い。; <_io.TextIOWrapper name='Jawiki Country.json.gz' encoding='utf-8'>
年: 28828
月: 12361
1: 8192
語: 6821
2: 5964
日本: 5440
3: 4740
こと: 4583
世界: 4147
4: 3996
5: 3836
ため: 3449
大統領: 3441
共和: 3432
政府: 3302
thumb: 3015
経済: 2932
6: 2926
8: 2843
7: 2823
